## Cartpole DQN

#### Import Dependencies

In [15]:
import random
import gym
import numpy as np
import os
from collections import deque
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam

#### Parameters

In [16]:
env = gym.make('CartPole-v0')
state_size = env.observation_space.shape[0]
action_size = env.action_space.n
print(state_size, action_size)

4 2


In [17]:
batch_size = 32
n_episodes = 500
output_dir = 'model_output/cartpole'

In [18]:
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

#### Define Agent

In [19]:
class DQNAgent:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        
        self.memory = deque(maxlen=2000)
        self.gamma = 0.95   # discount rate
        
        self.epsilon = 1.0  # exploration rate, 100% exploration at the beginning
        self.epsilon_decay = 0.995
        
        self.epsilon_min = 0.01 # 1% exploration at the end
        self.learning_rate = 0.001
        
        self.model = self.build_model()
    
    def build_model(self):
        # Neural Net for Deep-Q learning Model
        model = Sequential()
        model.add(Dense(24, input_dim=self.state_size, activation='relu'))
        model.add(Dense(24, activation='relu'))
        model.add(Dense(self.action_size, activation='linear'))
        model.compile(loss='mse', optimizer=Adam(learning_rate=self.learning_rate))
        return model
    
    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))
    
    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size) #explore
        act_values = self.model.predict(state)
        return np.argmax(act_values[0]) #exploit
    
    def replay(self, batch_size):
        minibatch = random.sample(self.memory, batch_size)
        
        for state, action, reward, next_state, done in minibatch:
            target = reward
            if not done:
                target = reward + self.gamma*np.amax(self.model.predict(next_state)[0])    
            target_f = self.model.predict(state)
            target_f[0][action] = target
            
            self.model.fit(state, target_f, epochs=1, verbose=0)
        
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay
    
    def load(self, name):
        self.model.load_weights(name)
        
    def save(self, name):
        self.model.save_weights(name)
        
    

In [20]:
agent = DQNAgent(state_size, action_size)

#### Interact with environment

In [21]:
done = False

for episode in range(n_episodes):
    state = env.reset()
    state = np.reshape(state, [1, state_size])
    
    for time in range(5000):
        env.render()
        action = agent.act(state)
        
        next_state, reward, done, _ = env.step(action)
        
        reward = reward if not done else -10
        
        next_state = np.reshape(state, [1, state_size])
        
        agent.remember(state, action, reward, next_state, done)
        
        state = next_state
        
        if done:
            print(f"episode: {episode}/{n_episodes}, score: {time}, e={agent.epsilon}")
            break
        
    if len(agent.memory) > batch_size:
        agent.replay(batch_size )
    
    if episode % 50 == 0:
        agent.save(output_dir + "weights_" + '{:04d}'.format(episode) + '.hdf5' )


episode: 0/500, score: 18, e=1.0
episode: 1/500, score: 35, e=1.0
1/1 [==============================] - 0s 26ms/step


Exception ignored in: <function Viewer.__del__ at 0x000002480AA348B0>
Traceback (most recent call last):
  File "c:\Users\azhar\miniconda3\envs\gym_bismillah\lib\site-packages\gym\envs\classic_control\rendering.py", line 185, in __del__
    self.close()
  File "c:\Users\azhar\miniconda3\envs\gym_bismillah\lib\site-packages\gym\envs\classic_control\rendering.py", line 101, in close
    self.window.close()
  File "c:\Users\azhar\miniconda3\envs\gym_bismillah\lib\site-packages\pyglet\window\win32\__init__.py", line 319, in close
    super(Win32Window, self).close()
  File "c:\Users\azhar\miniconda3\envs\gym_bismillah\lib\site-packages\pyglet\window\__init__.py", line 838, in close
    app.windows.remove(self)
  File "c:\Users\azhar\miniconda3\envs\gym_bismillah\lib\_weakrefset.py", line 114, in remove
    self.data.remove(ref(item))
KeyError: <weakref at 0x0000024814741170; to 'Win32Window' at 0x000002481232C070>


1/1 [==============================] - 0s 56ms/step
episode: 2/500, score: 14, e=0.995
1/1 [==============================] - 0s 49ms/step
episode: 3/500, score: 26, e=0.990025
1/1 [==============================] - 0s 53ms/step
episode: 4/500, score: 37, e=0.985074875
1/1 [==============================] - 0s 47ms/step
episode: 5/500, score: 30, e=0.9801495006250001
1/1 [==============================] - 0s 58ms/step
episode: 6/500, score: 18, e=0.9752487531218751
1/1 [==============================] - 0s 57ms/step
episode: 7/500, score: 26, e=0.9703725093562657
1/1 [==============================] - 0s 55ms/step
episode: 8/500, score: 15, e=0.9655206468094844
1/1 [==============================] - 0s 48ms/step
episode: 9/500, score: 20, e=0.960693043575437
1/1 [==============================] - 0s 49ms/step
episode: 10/500, score: 15, e=0.9558895783575597
1/1 [==============================] - 0s 54ms/step
episode: 11/500, score: 18, e=0.9511101304657719
1/1 [========================

KeyboardInterrupt: 